# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
print sum(data[data.race=='b'].call)

# number of callbacks for white-sounding names
print sum(data[data.race=='w'].call)

157.0
235.0


In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1,0,1,0,0,0,0,0,0,
1,b,1,3,3,6,0,1,1,0,316,...,1,0,1,0,0,0,0,0,0,
2,b,1,4,1,6,0,0,0,0,19,...,1,0,1,0,0,0,0,0,0,
3,b,1,3,4,6,0,1,0,1,313,...,1,0,1,0,0,0,0,0,0,
4,b,1,3,3,22,0,0,0,0,313,...,1,1,0,0,0,0,0,1,0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

<b>What test is appropriate for this problem? Does CLT apply?</b>

Because we are comparing two proportions, a 2-sample z-test would be appropriate for this problem.  For the CLT to apply, 3 conditions must be met : 1) the sample needs to be representative of the population, 2) the sample size should be smaller than 10% of the population, as a rule-of-thumb, and 3) the sample size should be sufficiently large (np > 10 and n(1-p) > 10, as a rule-of-thumb).  These data fit all 3 of these criteria, so the CLT does apply.


<b>What are the null and alternate hypotheses?</b>

Null hypothesis: The proportion of applicants with white-sounding names receiving callbacks is the same as the proportion of applicants with black-sounding names receiving callbacks

H<sub>0</sub>: p<sub>w</sub> = p<sub>b</sub>

Alternative hypothesis: The proportion of applicants with white-sounding names receiving callbacks is greater than the proportion of applicants with black-sounding names receiving callbacks

H<sub>a</sub>: p<sub>w</sub> > p<sub>b</sub>

In [5]:
w = data[data.race=='w']
b = data[data.race=='b']

<div class="span5 alert alert-success">
<p>Your solution to Q3 here</p>
</div>

In [6]:
# Calculate proportion of applications receiving callbacks with white-sounding and black-sounding names
nw = len(w.call)
nb = len(b.call)
pw = w.call.sum()/nw
pb = b.call.sum()/nb
p_diff = pw - pb
SE_pdiff = np.sqrt(pw*(1-pw)/nw + pb*(1-pb)/nb)

print('Proportion of callbacks for white-sounding names: {0:0.3f}'.format(pw))
print('Proportion of callbacks for black-sounding names: {0:0.3f}'.format(pb))
print('Difference in proportion of callbacks: {0:0.3f}'.format(p_diff))
print('Standard error for difference in proportion of callbacks: {0:0.3f}'
      .format(SE_pdiff))

alpha = 0.05 # define significance level

Proportion of call-backs for white-sounding names: 0.097
Proportion of call-backs for black-sounding names: 0.064
Difference in proportion of call-backs: 0.032
Standard error for difference in proportion of call-backs: 0.008


In [7]:
# Frequentist approach

z_star = 1.96 # For alpha = 0.05
margin_of_error = z_star * SE_pdiff
confidence_interval = [p_diff-margin_of_error, p_diff+margin_of_error]

print('Margin of Error: +-{0:0.4f}'.format(margin_of_error))
print('95% Confidence Interval: {0:0.4f} to {1:0.4f}'
      .format(confidence_interval[0],confidence_interval[1]))

z_stat = p_diff/SE_pdiff
print('Z-statistic: {:0.3f}'.format(z_stat))

p_val = 1 - stats.norm.cdf(z_stat)
print('p-value: {:f}'.format(p_val))

Margin of Error: +-0.0153
95% Confidence Interval: 0.0168 to 0.0473
Z-statistic: 4.116
p-value: 0.000019


In [54]:
# Bootstrap approach

calls = data.call.values
bs_pdiff = []
n_rep = 10000 # Number of bootstrap replicates
for _ in range(n_rep):
    bs_sample = np.random.permutation(calls)
    bs_pw = sum(bs_sample[:nw])/nw
    bs_pb = sum(bs_sample[nw:])/nb
    bs_pdiff.append(bs_pw-bs_pb)
    
p_val_bs = sum(bs_pdiff >= p_diff)/n_rep
print('p-value: {:f}'.format(p_val_bs))

p-value: 0.000000


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

<b>Write a story describing the statistical significance in the context or the original problem.</b>

The p-values for both approaches (bootstrap and frequentist) are lower than our significance level.  We can confidently reject the null hypothesis that white and black-sounding names receive the same callback rate and accept the alternative hypothesis that white-sounding names receive callbacks at a higher rate than black-sounding names.

<b>Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?</b>

No, the analysis does not suggest that race/name is the most important factor in callback success, but it does suggest that race/name has an impact on callback success when other factors are controlled (as the names were randomly assigned to applications).  Other factors could be equally or more important, such as education, experience, location, etc... or even certain combinations of factors.